In [1]:
%%capture
%load_ext autoreload
%autoreload 2
#Basic Imports
import os,sys
os.chdir("..")

from tqdm import tqdm,trange
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay
import torch
import pandas

from datasets.ssl_dataset import SSL_Dataset
from datasets.data_utils import get_data_loader
from utils import get_model_checkpoints
from utils import net_builder
from utils import manipulate_df

In [10]:
#Path to the runs to load
csv_folder = "/home/gabrielemeoni/project/SSLRS/test"
folder = "/scratch/fixmatch_results/nr_of_labels/eurosat_ms/"
sort_criteria = "numlabels" # Accepted net, numlabels

## Initialize parameters

In [11]:
checkpoints, run_args = get_model_checkpoints(folder)
if os.name == 'nt':
       [print(_.split("\\")[1]) for _ in checkpoints];
else:
       [print(_.split("/")[1]) for _ in checkpoints];

scratch
scratch
scratch
scratch
scratch
scratch


## Run all models

In [12]:
results = []
for checkpoint, args in zip(checkpoints,run_args):
    print("------------ RUNNING ", checkpoint, " -----------------")
    print(args)
    args["batch_size"] = 256
    args["data_dir"] = "./data/"
    args["use_train_model"] = False
    args["load_path"] = checkpoint
    
    checkpoint_path = os.path.join(args["load_path"])
    checkpoint = torch.load(checkpoint_path,map_location='cuda:0')
    load_model = (checkpoint["train_model"] if args["use_train_model"] else checkpoint["eval_model"])
    _net_builder = net_builder(args["net"],False,{})
    _eval_dset = SSL_Dataset(name=args["dataset"], train=False, data_dir=args["data_dir"], seed=args["seed"])
    eval_dset = _eval_dset.get_dset()
    net = _net_builder(num_classes=_eval_dset.num_classes, in_channels=_eval_dset.num_channels)
    net.load_state_dict(load_model)
    if torch.cuda.is_available():
        net.cuda()
    net.eval()

    
    

    eval_loader = get_data_loader(eval_dset, args["batch_size"], num_workers=1)
    label_encoding = _eval_dset.label_encoding
    inv_transf = _eval_dset.inv_transform

    
    print("------------ PREDICTING TESTSET -----------------")
    
    images, labels, preds = [],[],[]
    with torch.no_grad():
        for image, target in tqdm(eval_loader):
            image = image.type(torch.FloatTensor).cuda()
            logit = net(image)
            for idx,img in enumerate(image):
                images.append(inv_transf(img.transpose(0,2).cpu().numpy()).transpose(0,2).numpy())
            preds.append(logit.cpu().max(1)[1])
            labels.append(target)
    labels = torch.cat(labels).numpy()
    preds = torch.cat(preds).numpy()
    test_report = classification_report(labels, preds, target_names=label_encoding, output_dict=True)
    test_report["params"] = args
    results.append(test_report)

  0%|          | 0/10 [00:00<?, ?it/s]

------------ RUNNING  /scratch/fixmatch_results/nr_of_labels/eurosat_ms/FixMatch_archefficientnet-b2_batch32_confidence0.95_lr0.03_uratio7_wd0.0001_wu1.0_seed0_numlabels500_optSGD/model_best.pth  -----------------
{'dataset': 'eurosat_ms', 'net': 'efficientnet-b2', 'batch': 32, 'confidence': 0.95, 'lr': 0.03, 'uratio': 7, 'wd': 0.0001, 'wu': 1.0, 'seed': 0, 'numlabels': 500, 'opt': 'SGD', 'iterations': 260000}
Using not pretrained model efficientnet-b2 ...


  0%|          | 0/11 [00:00<?, ?it/s]

------------ PREDICTING TESTSET -----------------


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

------------ RUNNING  /scratch/fixmatch_results/nr_of_labels/eurosat_ms/FixMatch_archefficientnet-b2_batch32_confidence0.95_lr0.03_uratio7_wd0.0001_wu1.0_seed0_numlabels2000_optSGD/model_best.pth  -----------------
{'dataset': 'eurosat_ms', 'net': 'efficientnet-b2', 'batch': 32, 'confidence': 0.95, 'lr': 0.03, 'uratio': 7, 'wd': 0.0001, 'wu': 1.0, 'seed': 0, 'numlabels': 2000, 'opt': 'SGD', 'iterations': 368000}
Using not pretrained model efficientnet-b2 ...


  0%|          | 0/11 [00:00<?, ?it/s]

------------ PREDICTING TESTSET -----------------


100%|██████████| 11/11 [00:02<00:00,  4.68it/s]


In [13]:
import pandas as pd
big_df = pd.DataFrame()
pd.set_option('display.max_columns', None)
for result in results:
    params = result["params"]
    df = pd.DataFrame(result)
    df.drop(list(params.keys()),inplace=True)
    df.drop(["support","recall","precision"],inplace=True)
    for key,val in params.items():
        df[key] = val
    df = df.set_index("dataset")
    big_df = big_df.append(df)
print(big_df)
small_df = pd.DataFrame.from_dict(manipulate_df(big_df, folder,sort_criteria))
small_df.to_csv(csv_folder + "_test_results.csv")

            AnnualCrop    Forest  HerbaceousVegetation   Highway  Industrial  \
dataset                                                                        
eurosat_ms    0.935961  0.971332              0.898734  0.767544    0.934615   
eurosat_ms    0.453202  0.694239              0.559567  0.139706    0.797227   
eurosat_ms    0.954622  0.981758              0.944262  0.916836    0.964286   
eurosat_ms    0.537037  0.834983              0.748201  0.440415    0.828319   
eurosat_ms    0.981878  0.991653              0.968801  0.946058    0.970179   
eurosat_ms    0.969005  0.996656              0.967320  0.943089    0.971545   

             Pasture  PermanentCrop  Residential     River   SeaLake  \
dataset                                                                
eurosat_ms  0.898824       0.892704     0.955810  0.925253  0.984823   
eurosat_ms  0.382022       0.557241     0.794275  0.694639  0.887556   
eurosat_ms  0.945000       0.934426     0.978512  0.958580  0.984874   